In [21]:
import numpy as np
import pandas as pd
import os

import cv2
import matplotlib.pyplot as plt
from tslearn.preprocessing import TimeSeriesResampler
from pyts.image import GramianAngularField


from PIL import Image

# Image conversion using GASF and appending images with corresponding classes in arrays X and y respectively

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
path ='/content/drive/MyDrive/EEN-366 project dataset'

In [24]:
os.chdir(path)

In [25]:
pwd

'/content/drive/MyDrive/EEN-366 project dataset'

In [26]:
folder_list = ['LLL_csv_files', 'LLLG_csv_files', 'LL_csv_files', 'LLG_csv_files', 'SLG_csv_files']

In [27]:
X = []
y = []
N = 200

for i in range(len(folder_list)):
    print(folder_list[i])
    os.chdir(folder_list[i])


    for j in range(N):
        #print(j+1)

        #reading data
        filename = str(j+1)

        data = pd.read_csv(filename)
        Va = np.array(data['Unnamed: 0'])
        Vb = np.array(data['Unnamed: 1'])
        Vc = np.array(data['Unnamed: 2'])
        Ia = np.array(data['Unnamed: 3'])
        Ib = np.array(data['Unnamed: 4'])
        Ic = np.array(data['Unnamed: 5'])


        #scaling to [-1,1]
        Va = Va/max(Va)
        Vb = Vb/max(Vb)
        Vc = Vc/max(Vc)
        Ia = Ia/max(Ia)
        Ib = Ib/max(Ib)
        Ic = Ic/max(Ic)

        #resampling to reduce array size
        V = TimeSeriesResampler(sz=1000).fit_transform(Va)
        Va = np.squeeze(V)
        V = TimeSeriesResampler(sz=1000).fit_transform(Vb)
        Vb = np.squeeze(V)
        V = TimeSeriesResampler(sz=1000).fit_transform(Vc)
        Vc = np.squeeze(V)

        I = TimeSeriesResampler(sz=1000).fit_transform(Ia)
        Ia = np.squeeze(I)
        I = TimeSeriesResampler(sz=1000).fit_transform(Ib)
        Ib = np.squeeze(I)
        I = TimeSeriesResampler(sz=1000).fit_transform(Ic)
        Ic = np.squeeze(I)

        #reshaping the data
        Va = Va.reshape(np.size(Va))
        Va = np.array([Va])
        Vb = Vb.reshape(np.size(Vb))
        Vb = np.array([Vb])
        Vc = Vc.reshape(np.size(Vc))
        Vc = np.array([Vc])
        Ia = Ia.reshape(np.size(Ia))
        Ia = np.array([Ia])
        Ib = Ib.reshape(np.size(Ib))
        Ib = np.array([Ib])
        Ic = Ic.reshape(np.size(Ic))
        Ic = np.array([Ic])

        # Compute Gramian angular fields
        #for voltages
        gasf = GramianAngularField(method='summation')
        Va_gasf = gasf.fit_transform(Va)
        gasf = GramianAngularField(method='summation')
        Vb_gasf = gasf.fit_transform(Vb)
        gasf = GramianAngularField(method='summation')
        Vc_gasf = gasf.fit_transform(Vc)
        #for currents
        gasf = GramianAngularField(method='summation')
        Ia_gasf = gasf.fit_transform(Ia)
        gasf = GramianAngularField(method='summation')
        Ib_gasf = gasf.fit_transform(Ib)
        gasf = GramianAngularField(method='summation')
        Ic_gasf = gasf.fit_transform(Ic)


        #reshaping
        n = np.shape(Va_gasf)[1]
        Va_gasf = Va_gasf.reshape(n,n)
        Vb_gasf = Vb_gasf.reshape(n,n)
        Vc_gasf = Vc_gasf.reshape(n,n)

        Ia_gasf = Ia_gasf.reshape(n,n)
        Ib_gasf = Ib_gasf.reshape(n,n)
        Ic_gasf = Ic_gasf.reshape(n,n)

        #image  conversion
        img0 = Image.fromarray((Va_gasf* 255).astype(np.uint8))
        #print(np.shape(img0))
        img1 = Image.fromarray((Vb_gasf* 255).astype(np.uint8))
        img2 = Image.fromarray((Vc_gasf* 255).astype(np.uint8))
        img3 = Image.fromarray((Ia_gasf* 255).astype(np.uint8))
        img4 = Image.fromarray((Ib_gasf* 255).astype(np.uint8))
        img5 = Image.fromarray((Ic_gasf* 255).astype(np.uint8))

        #reducing image size
        WIDTH = 150
        HEIGHT = 150
        img0 = img0.resize((WIDTH, HEIGHT))
        img1 = img1.resize((WIDTH, HEIGHT))
        img2 = img2.resize((WIDTH, HEIGHT))
        img3 = img3.resize((WIDTH, HEIGHT))
        img4 = img4.resize((WIDTH, HEIGHT))
        img5 = img5.resize((WIDTH, HEIGHT))
        #plt.imshow(img0)

        #combining and stacking images
        img0 = np.array(img0)
        img1 = np.array(img1)
        img2 = np.array(img2)
        img3 = np.array(img3)
        img4 = np.array(img4)
        img5 = np.array(img5)


        imga = np.concatenate((img0, img3), axis=1)
        imgb = np.concatenate((img1, img4), axis=1)
        imgc = np.concatenate((img2, img5), axis=1)
        #print(np.shape(imga))
        imga = np.array(imga)
        imgb = np.array(imgb)
        imgc = np.array(imgc)

        imga = Image.fromarray((imga* 255).astype(np.uint8))
        imgb = Image.fromarray((imgb* 255).astype(np.uint8))
        imgc = Image.fromarray((imgc* 255).astype(np.uint8))


        img = np.stack((imga,imgb,imgc), axis = 2)
        X.append(img)
        y.append(i)

    os.chdir('..')






LLL_csv_files
LLLG_csv_files
LL_csv_files
LLG_csv_files
SLG_csv_files


In [28]:
img = X[0]
np.shape(img)

(150, 300, 3)

In [29]:
print(np.shape(X))
print(np.shape(y))

(1000, 150, 300, 3)
(1000,)


# Splitting data into test and train dataset

In [20]:
import keras
from sklearn.model_selection import train_test_split

In [37]:
X = np.array(X)
y = y.reshape(np.shape(y)[0])
y = np.array(y)
print(np.shape(X))
print(np.shape(y))

(1000, 150, 300, 3)
(1000,)


In [40]:
y_categorical = keras.utils.to_categorical(y, num_classes = 5)


In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.1, random_state=42)

In [66]:
print('X_train: ', len(X_train))
print('X_test:', len(X_test))



X_train:  900
X_test: 100


# CNN implementation with Transfer Learning

In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from pathlib import Path
import os


import keras
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.optimizers import Adam

#from keras.applications.vgg16 import VGG16, preprocess_input
#from keras.preprocessing.image import ImageDataGenerator

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten



In [69]:
base_model = ResNet50(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=None,
    include_top=False,
)

base_model.trainable = False

In [70]:
inputs = keras.Input(shape=(150, 300, 3))

scale_layer = keras.layers.Rescaling(scale=1/127.5, offset=-1)
x = scale_layer(inputs)

x = base_model(x, training=False)

x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)

outputs = keras.layers.Dense(5, activation = 'sigmoid')(x)

model = keras.Model(inputs, outputs)

In [71]:
model.summary(show_trainable=True)

Model: "model_2"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_6 (InputLayer)        [(None, 150, 300, 3)]     0         Y          
                                                                            
 rescaling_2 (Rescaling)     (None, 150, 300, 3)       0         Y          
                                                                            
 resnet50 (Functional)       (None, None, None, 2048   2358771   N          
                             )                         2                    
                                                                            
 global_average_pooling2d_2  (None, 2048)              0         Y          
  (GlobalAveragePooling2D)                                                  
                                                                            
 dropout_2 (Dropout)         (None, 2048)              0   

In [72]:
model.compile(loss=keras.metrics.categorical_crossentropy, optimizer=keras.optimizers.Adam(learning_rate = 0.001), metrics=['accuracy'])

In [73]:
#training
epochs = 25
history =  model.fit(X_train, y_train, epochs=epochs)

Epoch 1/25
29/29 [==============================] - 180s 6s/step - loss: 1.6668 - accuracy: 0.2344
Epoch 2/25
29/29 [==============================] - 171s 6s/step - loss: 1.5617 - accuracy: 0.2833
Epoch 3/25
29/29 [==============================] - 171s 6s/step - loss: 1.4906 - accuracy: 0.3256
Epoch 4/25
29/29 [==============================] - 171s 6s/step - loss: 1.4550 - accuracy: 0.3700
Epoch 5/25
29/29 [==============================] - 173s 6s/step - loss: 1.4447 - accuracy: 0.3600
Epoch 6/25
29/29 [==============================] - 168s 6s/step - loss: 1.4163 - accuracy: 0.3778
Epoch 7/25
29/29 [==============================] - 173s 6s/step - loss: 1.4224 - accuracy: 0.3689
Epoch 8/25
29/29 [==============================] - 170s 6s/step - loss: 1.3731 - accuracy: 0.4089
Epoch 9/25
29/29 [==============================] - 172s 6s/step - loss: 1.3677 - accuracy: 0.4022
Epoch 10/25
29/29 [==============================] - 171s 6s/step - loss: 1.3783 - accuracy: 0.3744
Epoch 11/

# Feature extraction

In [2]:
#feature extraction
index = 5
feature_extractor = keras.Model(
    inputs=model.inputs,
    outputs=model.get_layer(index=index).output,
)

NameError: name 'keras' is not defined

In [ ]:
X_new = feature_extractor(X)

In [ ]:
X_new.shape

# SVM classifier

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state = 5102023)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,confusion_matrix

In [ ]:
model = SVC(kernel = 'rbf')

In [ ]:
model.fit(X_train,y_train)

In [ ]:
predictedtrain = model.predict(X_train)
accuracy_score(y_train,predictedtrain)

In [ ]:
predictedtest = model.predict(X_test)
accuracy_score(y_test,predictedtest)